# 坡位分类(按照坡度和相对位置)

In [1]:
import arcpy
from arcpy.sa import *
from arcpy import env
from arcpy.conversion import *
from arcpy.management import *
from arcpy.da import *
import os
import pandas as pd
import numpy as np
from arcpy.sa import *
import math

In [2]:
# 分析函数

# 取消并行处理
def disable_parallel_processing(func):
    def wrapper(*args, **kwargs):
        with arcpy.EnvManager(parallelProcessingFactor="0"):
            return func(*args, **kwargs)
    return wrapper

# 填洼
@disable_parallel_processing
def fill_dem(dem_raster):
    """参数默认为最佳"""
    fill_raster = Fill(dem_raster)
    print("fill done!")
    return fill_raster
# 流向
@disable_parallel_processing
def flowdir_dem(dem_raster,dir_index):
    """"D8算法,dir_index{1:正常流动,0:强制外向流动}"""
    if dir_index:
        flowdir_raster = FlowDirection(dem_raster,"NORMAL",None,"D8")
        return flowdir_raster
    else:
        flowdir_raster = FlowDirection(dem_raster,"FORCE", None, "D8")
        return flowdir_raster
# 流量
@disable_parallel_processing
def acc_dem(fd_raster):
    """输入流向类型D8"""
    acc_raster = FlowAccumulation(fd_raster,None,"FLOAT","D8")
    return acc_raster

# 基于焦点的正负地形
def pos_neg_dem(dem_raster,focus_size,index_):
    """focus_size:统计焦点,Test:if:(true:1正),else:0负"""
    if index_:
        pos_raster = Test(dem_raster - FocalStatistics(dem_raster,NbrRectangle(focus_size,focus_size,"CELL"),"MEAN","DATA"),"value>0")
        return pos_raster
    else:
        neg_raster = Test(dem_raster - FocalStatistics(dem_raster,NbrRectangle(focus_size,focus_size,"CELL"),"MEAN","DATA"),"value<0")
        return neg_raster
# 基于指定值的负地形
def digital_dem(dem_raster,elevation_value):
    """基于指定值于DEM作差得出的栅格"""
    return elevation_value-arcpy.Raster(dem_raster)
# 坡度
@disable_parallel_processing
def get_slope(dem_raser):
    """取消并行处理"""
    slope_raster = Slope(dem_raser,"DEGREE",1,"PLANAR","METER")
    return slope_raster
# 坡向
@disable_parallel_processing
def get_asp(dem_raster):
    """取消并行处理"""
    asp_raster = Aspect(dem_raster,"PLANAR", "METER", "GEODESIC_AZIMUTHS")
    return asp_raster
# 计算曲率
@disable_parallel_processing
def get_cur(dem_raster,count):
    """count:{0:曲率,1:[剖面曲率,平面曲率]}"""
    if count:
        asp_raster = Curvature(dem_raster,1,"profile_curve","plan_curve")
        return asp_raster
    else:
        asp_raster = Curvature(dem_raster,1,None,None)
        return asp_raster
# arcpy栅格众数滤波
@disable_parallel_processing
def mode_filter(in_raster,filter_size="FOUR",filter_method="MAJORITY"):
    """栅格众数滤波,filter_size:滤波窗口大小,filter_method:滤波方法"""
    result_raster = MajorityFilter(in_raster, filter_size, filter_method) # 默认为4邻域 众数滤波  可选为8邻域 半数滤波
    return result_raster
    
# arcpy栅格边界清理
@disable_parallel_processing
def clean_boundary(in_raster,sort_method="NO_SORT",clean_method="TWO_WAY"):
    """栅格边界清理,sort_method:排序方法,clean_method:清理方法"""
    result_raster = BoundaryClean(in_raster, "NO_SORT", "TWO_WAY") # 默认为不排序 两边清理 可选为排序 一边清理
    return result_raster
# arcpy栅格区域分组
@disable_parallel_processing
def region_group(in_raster,group_size="FOUR",method="WITHIN",add_link="ADD_LINK",clear=None):
    """栅格区域分组,group_size:分组大小,method:分组方法,add_link:添加链接,clear:排除的值"""
    result_raster = RegionGroup(in_raster, group_size, method, add_link, clear) # 默认为4邻域 分组方法为WITHIN 可选为8邻域 分组方法为CROSS
    return result_raster
# arcpy栅格按属性提取
@disable_parallel_processing
def extract_by_attributes(in_raster,where_clause="count >= 100"):
    """栅格按属性提取,where_clause:属性条件"""
    result_raster = ExtractByAttributes(in_raster, where_clause) # 默认为count >= 200
    return result_raster

# arcpy栅格Nibble
@disable_parallel_processing
def nibble(in_raster,mask_raster, nibble_mask="ALL_VALUES", nibble_values="PRESERVE_NODATA"):
    """栅格Nibble,nibble_mask:掩膜,nibble_values:填充值"""
    result_raster = Nibble(in_raster, mask_raster,nibble_mask, nibble_values) # 默认为ALL_VALUES PRESERVE_NODATA 可选为DATA_ONLY PRESERVE_DATA
    return result_raster

# arcpy栅格查找表
@disable_parallel_processing
def lookup(in_raster,lookup_field="LINK"):
    """栅格查找表,lookup_field:查找字段"""
    result_raster = Lookup(in_raster, lookup_field) # 默认为LINK
    return result_raster

# 栅格转面函数
def raster_to_polygon(in_raster,out_polygon):
    arcpy.conversion.RasterToPolygon(
        in_raster=in_raster,
        out_polygon_features=out_polygon,
        simplify="SIMPLIFY",
        raster_field="Value",
        create_multipart_features="SINGLE_OUTER_PART",
        max_vertices_per_feature=None
    )


# 平滑面函数
def smooth_polygon(in_polygon,out_polygon):
    arcpy.cartography.SmoothPolygon(
        in_features=in_polygon,
        out_feature_class=out_polygon,
        algorithm="PAEK",
        tolerance="50 Meters",
        endpoint_option="FIXED_ENDPOINT",
        error_option="RESOLVE_ERRORS",
        in_barriers=None
    )


In [3]:
def calc_class_raster(dem_raster, slope_raster, focus_size):
    """
    根据规则计算DEM栅格的TPI并分类：
    tpi > 1 sd : 山脊(6)，
    0.5 sd <= tpi <= 1 sd : 坡上(5)，
    -0.5 sd < tpi < 0.5 sd and slope > 8: 中坡(4)，(默认5度)
    -0.5 sd < tpi < 0.5 sd and slope <= 8: 平坡(3)，(默认5度)
    -1 sd < tpi <= -0.5 sd : 坡下(2)，
    tpi < -1 sd : 山谷(1)。
    """
    # DEM
    dem_raster = Raster(dem_raster)
    # 计算局部均值
    mean_raster = FocalStatistics(
        dem_raster, NbrRectangle(focus_size, focus_size, "CELL"), "MEAN", "DATA"
    )
    # 输出日志
    print("计算局部均值完成")
    # 计算局部标准差
    std_raster = FocalStatistics(
        dem_raster, NbrRectangle(focus_size, focus_size, "CELL"), "STD", "DATA"
    )
    # 输出日志
    print("计算局部标准差完成")
    # 计算TPI
    tpi_raster = dem_raster - mean_raster
    # 输出日志
    print("计算TPI完成")

    # 根据规则分类
    result_raster = Con(tpi_raster > std_raster, 6,
                        Con((tpi_raster > 0.5 * std_raster) & (tpi_raster <= std_raster), 5,
                            Con((tpi_raster > -0.5 * std_raster) & (tpi_raster < 0.5 * std_raster) & (slope_raster > 5), 4,
                                Con((tpi_raster > -0.5 * std_raster) & (tpi_raster < 0.5 * std_raster) & (slope_raster <= 5), 3,
                                    Con((tpi_raster > -std_raster) & (tpi_raster <= -0.5 * std_raster), 2,
                                        Con(tpi_raster < -std_raster, 1))))))
    # 输出日志
    print("计算DEM栅格的TPI并分类完成")
    
    return result_raster

In [4]:
# 焦点统计函数
# 获取均值
def get_mean(dem_raster,focus_size):
    dem_raster = Raster(dem_raster)
    mean_raster = FocalStatistics(
        dem_raster, NbrRectangle(focus_size, focus_size, "CELL"), "MEAN", "DATA"
    )
    return mean_raster

# 获取标准差
def get_std(dem_raster,focus_size):
    dem_raster = Raster(dem_raster)
    std_raster = FocalStatistics(
        dem_raster, NbrRectangle(focus_size, focus_size, "CELL"), "STD", "DATA"
        )
    return std_raster
# 获取TPI
def get_tpi(dem_raster,mean_raster,focus_size):
    dem_raster = Raster(dem_raster)
    tpi_raster = dem_raster - mean_raster
    return tpi_raster

In [5]:
# 设定工作环境
work_path = r"C:\Users\Runker\Desktop\slope_temp"
os.makedirs(work_path, exist_ok=True)
env.workspace=work_path
dem_file_path  = r"F:\ArcgisData\m5\wc\wc_mosaic_dem.tif"
slope_name= 'wc_slope'
result_name= 'wc_slope_101'
dem_name = 'wc'

In [6]:
# 允许覆盖
env.overwriteOutput = True

In [ ]:
# 先填洼
fill_raster = fill_dem(dem_file_path)
fill_raster.save(f'{dem_name}_fill.tif')

In [8]:
# 得到坡度
slope_raster  = get_slope(fill_raster)
slope_raster.save(f'{slope_name}.tif')

In [ ]:
# 得到分类结果
result = calc_class_raster(fill_raster,slope_raster,101)

In [10]:
result.save(f'{result_name}.tif')


In [11]:
raster_path = f"{work_path}\{result_name}.tif"
result_path = f"{work_path}\{result_name}_result.tif"

In [12]:
# --->>>概化栅格
# 读取栅格 并整型
indoor_raster = Raster(raster_path)
int_raster = Int(indoor_raster)
# 1.众数滤波
# raster_majority_filter = mode_filter(int_raster,"FOUR","MAJORITY")
raster_majority_filter = mode_filter(int_raster,"EIGHT","MAJORITY")

# 2.边界清理
raster_boundary_clean = clean_boundary(raster_majority_filter,"NO_SORT","TWO_WAY")

# 3.区域分组
raster_region_group = region_group(raster_boundary_clean,"FOUR","WITHIN","ADD_LINK",None)
# raster_region_group = region_group(raster_boundary_clean,"EIGHT","WITHIN","ADD_LINK",None)

# 4.提取区域边界
raster_region_boundary = extract_by_attributes(raster_region_group,"COUNT > 200")

# 5.查找表
raster_lookup = lookup(raster_region_group,"LINK")

# 5.蚕食
raster_nibble = nibble(raster_lookup,raster_region_boundary,"ALL_VALUES","PRESERVE_NODATA")
# 6.区域并保存
Int(raster_nibble).save(result_path)

In [13]:
# 栅格转面
out_polygon = result_path.replace(".tif",".shp")
raster_to_polygon(result_path,out_polygon)

In [14]:
# 平滑面
out_smooth_polygon = out_polygon.replace(".shp","_smooth.shp")
smooth_polygon(out_polygon,out_smooth_polygon)